<a target="_blank" href="https://colab.research.google.com/github/jingjieyeo/lammps_on_colab/blob/main/lammps_on_colab.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
# First, get all the libraries and executables that we will need

# We start with installing Anaconda
!pip install -q condacolab
import condacolab
condacolab.install()

# Then we get the packages needed for LAMMPS
!conda install -c conda-forge lammps

# Get packages needed for MD trajectory analyses and visualization
!pip install mdtraj
!pip install py3dmol 
!pip install lammps-logfile

In [ ]:
# Mount your Google Drive
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# Create a working directory, possible to skip if you do not want to save your files
!mkdir -p /content/drive/MyDrive/lammps_on_colab
%cd /content/drive/MyDrive/lammps_on_colab

In [ ]:
# Once everything is installed, load the libraries and files
import mdtraj as md
import py3Dmol
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lammps_logfile as lmp_log

In [ ]:
# Remove existing files, run only if needed
!rm -rf *.xyz *.in *.out *.lammps

In [ ]:
# Get the LAMMPS input file that we need, uncomment the relevant files

# Argon modelled using LJ interactions, normalized units
!wget -O argon_lj.in https://raw.githubusercontent.com/jingjieyeo/lammps_on_colab/main/examples/lennard_jones_argon/argon_lj.in

# Mixture of Argon and Neon using LJ interactions, normalized units
#!wget -O mix_lj.in https://raw.githubusercontent.com/jingjieyeo/lammps_on_colab/main/examples/lennard_jones_mixtures/mix_lj.in

# Aluminum using LJ interactions, real units
#!wget -O al_lj.in https://raw.githubusercontent.com/jingjieyeo/lammps_on_colab/main/examples/lennard_jones_aluminum/al_lj.in

# Tensile test on Aluminum using LJ interactions, real units
#!wget -O al_lj_tensile.in https://raw.githubusercontent.com/jingjieyeo/lammps_on_colab/main/examples/lennard_jones_aluminum/al_lj_tensile.in

# Polymer example using PMMA
#!wget -O pmma.zip https://raw.githubusercontent.com/jingjieyeo/lammps_on_colab/main/examples/pmma/pmma.zip
#!unzip pmma.zip -d ./pmma
#%cd pmma

In [ ]:
# Run LAMMPS
# Argon
!mpirun --allow-run-as-root lmp_mpi -in argon_lj.in

# Mixture
#!mpirun --allow-run-as-root lmp_mpi -in mix_lj.in

# Aluminum
#!mpirun --allow-run-as-root lmp_mpi -in al_lj.in

# Tensile test on Aluminum
#!mpirun --allow-run-as-root lmp_mpi -in al_lj_tensile.in

In [ ]:
# Have a look at the output from LAMMPS
!cat log.lammps

In [ ]:
# Plot LAMMPS thermo output
log = lmp_log.File("./log.lammps")

# Plot the variation of temperature with time
x = log.get("Step")
y = log.get("Temp")

plt.figure(figsize=(10, 5))
plt.title("Temperature")
plt.xlabel("No. of Steps")
plt.ylabel("Temperature")
plt.plot(x, y)
plt.show()

In [ ]:
# Plot the variation of temperature with time
pe = log.get("PotEng")
ke = log.get("KinEng")
te = log.get("TotEng")

plt.figure(figsize=(10, 5))
plt.title("Energy")
plt.xlabel("No. of Steps")
plt.ylabel("Energy")
plt.plot(x, pe, label="Potential Energy")
plt.plot(x, ke, label="Kinetic Energy")
plt.plot(x, te, label="Total Energy")
plt.legend()
plt.show()

In [ ]:
# Create the viewer
viewer = py3Dmol.view(width=400, height=400)

# Add all models to the viewer and set the frames for animation
viewer.addModelsAsFrames(open('equil.xyz', 'r').read(), 'xyz')
viewer.setBackgroundColor('white')
viewer.setStyle({'sphere': {'color':'green', "scale": 0.25}}) 

viewer.zoomTo()

# Set animation parameters
viewer.animate({'loop': 'forward'})

# Display the viewer
viewer.show()